# Implementing Gradient Descend

## Data preparation

In [2]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

,gre,gpa,rank_1,rank_2,rank_3,rank_4
209,-0.066657,0.289305,0,1,0,0
280,0.625884,1.445476,0,1,0,0
33,1.837832,1.603135,0,0,1,0
210,1.318426,-0.131120,0,0,0,1
93,-0.066657,-1.208461,0,1,0,0
84,-0.759199,0.552071,0,0,1,0
329,-0.759199,-1.208461,0,0,0,1
94,0.625884,0.131646,0,1,0,0
266,-0.239793,-0.393886,0,0,0,1
126,0.106478,0.394412,1,0,0,0


## Gradient Descent

In [4]:
import numpy as np
#from data_prep import features, targets, features_test, targets_test


def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    """
    Calculate the partial derivative of the sigmoid function
    """
    sig = sigmoid(x)
    return sig * (1 - sig)

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Calculate the output
        h = np.dot(x,weights)
        output = sigmoid(h)

        # Calculate the error
        error = y - output

        # Calculate the error term
        error_term = error * sigmoid_prime(h)

        # Calculate the change in weights for this sample
        # and add it to the total weight change
        del_w += learnrate * error_term * x

    # Update weights using the learning rate and the average change in weights
    weights += del_w / weights.shape

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

[-0.06665712  0.28930534  0.          1.          0.          0.        ]
('Train loss: ', 0.2637214310177573)
[-0.06665712  0.28930534  0.          1.          0.          0.        ]
('Train loss: ', 0.26326122098494759)
[-0.06665712  0.28930534  0.          1.          0.          0.        ]
('Train loss: ', 0.26280631065522386)
[-0.06665712  0.28930534  0.          1.          0.          0.        ]
('Train loss: ', 0.26235683847143881)
[-0.06665712  0.28930534  0.          1.          0.          0.        ]
('Train loss: ', 0.26191293860928172)
[-0.06665712  0.28930534  0.          1.          0.          0.        ]
('Train loss: ', 0.26147474077977378)
[-0.06665712  0.28930534  0.          1.          0.          0.        ]
('Train loss: ', 0.26104237004216396)
[-0.06665712  0.28930534  0.          1.          0.          0.        ]
('Train loss: ', 0.26061594662785459)
[-0.06665712  0.28930534  0.          1.          0.          0.        ]
('Train loss: ', 0.260195585775